# CSP Paper Trading Notebook

Paper-trades cash-secured puts using Alpaca (paper) with VIX gating, equity/option filters, and configurable limit/market entry and exit.

## 1. Setup & Configuration

In [ ]:
# Install required packages (run once)
# !pip install alpaca-py yfinance pandas numpy python-dotenv py_vollib

In [1]:
import os
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple
from datetime import datetime, date, timedelta
from enum import Enum
import pandas as pd
import numpy as np
import yfinance as yf
from dotenv import load_dotenv

from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetOptionContractsRequest, MarketOrderRequest, LimitOrderRequest
from alpaca.trading.enums import AssetStatus, OrderSide, TimeInForce
from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.data.requests import OptionSnapshotRequest

load_dotenv()
print("[Imports] OK — alpaca-py, yfinance, pandas, numpy, dotenv loaded.")

Imports successful!


In [2]:
@dataclass
class StrategyConfig:
    """Central configuration for CSP paper trading. Order-type toggles control entry/exit behavior."""
    # ==================== UNIVERSE & CAPITAL ====================
    ticker_universe: List[str] = field(default_factory=lambda: ['AAPL', 'MSFT', 'GOOG'])
    num_tickers: int = 10
    starting_cash: float = 50_000

    # ==================== ORDER TYPE TOGGLES ====================
    use_limit_entry: bool = True   # False = market entry
    use_limit_exit: bool = False    # True = limit exit (e.g. at bid); False = market exit

    # ==================== VIX REGIME ====================
    vix_deployment_rules: Dict[Tuple[float, float], float] = field(default_factory=lambda: {
        (0, 15): 1.0, (15, 20): 0.8, (20, 25): 0.2, (25, float('inf')): 0.0
    })

    # ==================== EQUITY FILTER ====================
    sma_periods: List[int] = field(default_factory=lambda: [8, 20, 50])
    rsi_period: int = 14
    rsi_lower: int = 30
    rsi_upper: int = 70
    bb_period: int = 50
    bb_std: float = 1.0
    sma_trend_lookback: int = 3
    history_days: int = 60

    # ==================== OPTIONS FILTER ====================
    min_daily_return: float = 0.15   # 0.15% daily on strike (as decimal 0.0015)
    max_strike_pct: float = 0.98
    min_strike_pct: float = 0.85
    delta_min: float = 0.15
    delta_max: float = 0.40
    max_dte: int = 10
    min_dte: int = 1

    # ==================== RISK ====================
    delta_stop_multiplier: float = 2.0
    stock_drop_stop_pct: float = 0.05
    vix_spike_multiplier: float = 1.15
    early_exit_buffer: float = 0.15

    # ==================== OPERATIONAL ====================
    poll_interval_seconds: int = 60
    paper_trading: bool = True
    dry_run: bool = True   # If True, no real orders; log only

    def get_deployment_multiplier(self, vix: float) -> float:
        for (lower, upper), mult in self.vix_deployment_rules.items():
            if lower <= vix < upper:
                return mult
        return 0.0

    def get_deployable_cash(self, vix: float) -> float:
        return self.starting_cash * self.get_deployment_multiplier(vix)


config = StrategyConfig()
print("[Config] Entry:", "limit" if config.use_limit_entry else "market", "| Exit:", "limit" if config.use_limit_exit else "market", "| Dry run:", config.dry_run)
print(f"  Universe: {config.ticker_universe} | history_days: {config.history_days} | min_daily_return: {config.min_daily_return}% | delta: [{config.delta_min}, {config.delta_max}]")

Config loaded. Entry: limit | Exit: market | Dry run: True


## 2. Alpaca Clients

In [3]:
class AlpacaClientManager:
    """Manages Alpaca data and trading clients. Credentials from env (ALPACA_API_KEY, ALPACA_SECRET_KEY)."""
    def __init__(self, paper: bool = True):
        self.paper = paper
        self.api_key = os.getenv('ALPACA_API_KEY')
        self.secret_key = os.getenv('ALPACA_SECRET_KEY')
        if not self.api_key or not self.secret_key:
            raise ValueError("Set ALPACA_API_KEY and ALPACA_SECRET_KEY in .env")
        self._data_client = None
        self._trading_client = None
        self._option_data_client = None

    @property
    def data_client(self) -> StockHistoricalDataClient:
        if self._data_client is None:
            self._data_client = StockHistoricalDataClient(api_key=self.api_key, secret_key=self.secret_key)
        return self._data_client

    @property
    def trading_client(self) -> TradingClient:
        if self._trading_client is None:
            self._trading_client = TradingClient(api_key=self.api_key, secret_key=self.secret_key, paper=self.paper)
        return self._trading_client

    @property
    def option_data_client(self):
        if self._option_data_client is None:
            self._option_data_client = OptionHistoricalDataClient(api_key=self.api_key, secret_key=self.secret_key)
        return self._option_data_client

    def get_account_info(self) -> dict:
        acc = self.trading_client.get_account()
        return {'cash': float(acc.cash), 'buying_power': float(acc.buying_power), 'portfolio_value': float(acc.portfolio_value)}


try:
    alpaca = AlpacaClientManager(paper=config.paper_trading)
    info = alpaca.get_account_info()
    print("[Alpaca] OK. Cash:", f"${info['cash']:,.2f}", "| Buying power:", f"${info['buying_power']:,.2f}", "| Portfolio value:", f"${info['portfolio_value']:,.2f}")
except Exception as e:
    print("[Alpaca] Not configured:", e)
    alpaca = None

Alpaca OK. Cash: $50,000.00


## 3. Data Layer (VIX, Equity, Options)

In [4]:
class VixDataFetcher:
    """VIX via yfinance ^VIX."""
    def __init__(self):
        self._ticker = yf.Ticker("^VIX")
        self._cache, self._cache_time = {}, None
        self._ttl = timedelta(minutes=1)

    def get_current_vix(self) -> float:
        if self._cache_time and datetime.now() - self._cache_time < self._ttl and 'current' in self._cache:
            return self._cache['current']
        daily = self._ticker.history(period='5d')
        if daily.empty:
            raise RuntimeError("No VIX data")
        vix = float(daily['Close'].iloc[-1])
        self._cache['current'], self._cache_time = vix, datetime.now()
        return vix

    def get_last_session(self) -> dict:
        h = self._ticker.history(period='5d')
        if h.empty:
            raise RuntimeError("No VIX history")
        r = h.iloc[-1]
        return {'session_date': h.index[-1], 'open': float(r['Open']), 'close': float(r['Close'])}

    def check_vix_stop_loss(self, reference_vix: float, multiplier: float) -> dict:
        current = self.get_current_vix()
        threshold = reference_vix * multiplier
        return {'triggered': current >= threshold, 'current_vix': current, 'threshold': threshold}


vix_fetcher = VixDataFetcher()
vix_val = vix_fetcher.get_current_vix()
session = vix_fetcher.get_last_session()
print("[VIX] Current VIX:", vix_val)
print(f"  Last session: open={session.get('open')} close={session.get('close')}")
print(f"  Deployable (config): ${config.get_deployable_cash(vix_val):,.0f}")

VIX: 17.440000534057617


In [5]:
class EquityDataFetcher:
    """Equity bars from Alpaca."""
    def __init__(self, alpaca_mgr):
        self.client = alpaca_mgr.data_client

    def get_close_history(self, symbols: List[str], days: int = 60) -> Dict[str, pd.Series]:
        end_d = datetime.now()
        start_d = end_d - timedelta(days=int(days * 1.5))
        req = StockBarsRequest(symbol_or_symbols=symbols, timeframe=TimeFrame.Day, start=start_d, end=end_d)
        bars = self.client.get_stock_bars(req)
        out = {}
        for sym in symbols:
            if sym in bars.data:
                b = bars.data[sym]
                out[sym] = pd.Series([x.close for x in b], index=[x.timestamp for x in b]).tail(days)
        return out

    def get_current_prices(self, symbols: List[str]) -> Dict[str, float]:
        h = self.get_close_history(symbols, days=5)
        return {s: float(p.iloc[-1]) for s, p in h.items() if len(p) > 0}


if alpaca:
    equity_fetcher = EquityDataFetcher(alpaca)
    print("[EquityDataFetcher] OK — Alpaca equity client ready.")
    # Quick sanity: fetch close history for universe
    try:
        close_hist = equity_fetcher.get_close_history(config.ticker_universe[:3], 5)
        prices = equity_fetcher.get_current_prices(list(close_hist.keys()))
        print(f"  Sample: {len(close_hist)} symbols, current prices = {prices}")
    except Exception as e:
        print(f"  Sample fetch error: {e}")
else:
    equity_fetcher = None
    print("[EquityDataFetcher] Skipped — Alpaca not configured.")

In [6]:
from alpaca.trading.enums import ContractType

RISK_FREE_RATE = 0.04

try:
    from py_vollib.black_scholes.implied_volatility import implied_volatility
    from py_vollib.black_scholes.greeks.analytical import delta as bs_delta
    _has_vollib = True
except ImportError:
    _has_vollib = False


class GreeksCalculator:
    """IV and delta via py_vollib (optional)."""
    def __init__(self, r: float = RISK_FREE_RATE):
        self.r = r

    def compute_greeks(self, option_price: float, stock_price: float, strike: float, dte: int, option_type: str = 'put') -> dict:
        if not _has_vollib or not all([np.isfinite(option_price), option_price > 0, stock_price > 0, strike > 0, dte > 0]):
            return {'iv': None, 'delta': None, 'delta_abs': None}
        t = dte / 365.0
        flag = 'p' if option_type == 'put' else 'c'
        try:
            iv = implied_volatility(option_price, stock_price, strike, t, self.r, flag)
            iv = iv if np.isfinite(iv) and iv > 0 else None
        except Exception:
            iv = None
        delta = None
        if iv:
            try:
                delta = bs_delta(flag, stock_price, strike, t, self.r, iv)
                delta = delta if np.isfinite(delta) else None
            except Exception:
                pass
        return {'iv': iv, 'delta': delta, 'delta_abs': abs(delta) if delta else None}


@dataclass
class OptionContract:
    symbol: str
    underlying: str
    contract_type: str
    strike: float
    expiration: date
    dte: int
    bid: float
    ask: float
    mid: float
    stock_price: float
    delta: Optional[float] = None
    implied_volatility: Optional[float] = None

    @property
    def premium(self) -> float:
        return self.bid

    @property
    def premium_per_day(self) -> float:
        return self.premium / self.dte if self.dte > 0 else 0.0

    @property
    def collateral_required(self) -> float:
        return self.strike * 100

    @property
    def daily_return_on_collateral(self) -> float:
        if self.strike <= 0 or self.dte <= 0:
            return 0.0
        return self.premium_per_day / self.strike

    @property
    def delta_abs(self) -> Optional[float]:
        return abs(self.delta) if self.delta is not None else None


greeks_calc = GreeksCalculator()
print("[Greeks/OptionContract] GreeksCalculator and OptionContract defined. py_vollib available:", _has_vollib)

In [7]:
class OptionsDataFetcher:
    """Options chain and snapshots from Alpaca."""
    def __init__(self, alpaca_mgr):
        self.trading_client = alpaca_mgr.trading_client
        self.data_client = alpaca_mgr.option_data_client

    def get_option_contracts(self, underlying: str, contract_type: str, min_dte: int, max_dte: int,
                            min_strike: Optional[float], max_strike: Optional[float]) -> List[dict]:
        today = date.today()
        min_exp = today + timedelta(days=min_dte)
        max_exp = today + timedelta(days=max_dte)
        params = {'underlying_symbols': [underlying], 'status': AssetStatus.ACTIVE,
                  'type': ContractType.PUT if contract_type == 'put' else ContractType.CALL,
                  'expiration_date_gte': min_exp, 'expiration_date_lte': max_exp}
        if min_strike is not None:
            params['strike_price_gte'] = str(min_strike)
        if max_strike is not None:
            params['strike_price_lte'] = str(max_strike)
        req = GetOptionContractsRequest(**params)
        resp = self.trading_client.get_option_contracts(req)
        out = []
        if resp.option_contracts:
            for c in resp.option_contracts:
                out.append({'symbol': c.symbol, 'underlying': c.underlying_symbol, 'strike': float(c.strike_price),
                            'expiration': c.expiration_date, 'contract_type': contract_type})
        return out

    def get_option_snapshots(self, option_symbols: List[str]) -> Dict[str, dict]:
        if not option_symbols:
            return {}
        all_snap = {}
        for i in range(0, len(option_symbols), 100):
            chunk = option_symbols[i:i + 100]
            try:
                req = OptionSnapshotRequest(symbol_or_symbols=chunk)
                snapshots = self.data_client.get_option_snapshot(req)
                for sym, snap in snapshots.items():
                    g = snap.greeks
                    q = snap.latest_quote
                    all_snap[sym] = {
                        'bid': float(q.bid_price) if q and q.bid_price else 0.0,
                        'ask': float(q.ask_price) if q and q.ask_price else 0.0,
                        'delta': float(g.delta) if g and g.delta else None,
                        'implied_volatility': float(snap.implied_volatility) if snap.implied_volatility else None,
                    }
            except Exception as e:
                print("Snapshot error:", e)
        return all_snap

    def get_puts_chain(self, underlying: str, stock_price: float, cfg: StrategyConfig) -> List[OptionContract]:
        min_strike = stock_price * 0.70
        max_strike = stock_price * cfg.max_strike_pct
        contracts = self.get_option_contracts(underlying, 'put', cfg.min_dte, cfg.max_dte, min_strike, max_strike)
        if not contracts:
            return []
        symbols = [c['symbol'] for c in contracts]
        snapshots = self.get_option_snapshots(symbols)
        today = date.today()
        result = []
        min_daily_pct = cfg.min_daily_return / 100.0
        for c in contracts:
            sym = c['symbol']
            snap = snapshots.get(sym, {})
            bid, ask = snap.get('bid', 0.0), snap.get('ask', 0.0)
            if bid <= 0:
                continue
            dte = (c['expiration'] - today).days
            delta = snap.get('delta')
            if delta is None:
                g = greeks_calc.compute_greeks((bid + ask) / 2, stock_price, c['strike'], dte, 'put')
                delta = g.get('delta')
            opt = OptionContract(symbol=sym, underlying=underlying, contract_type='put', strike=c['strike'],
                                 expiration=c['expiration'], dte=dte, bid=bid, ask=ask, mid=(bid + ask) / 2,
                                 stock_price=stock_price, delta=delta, implied_volatility=snap.get('implied_volatility'))
            if opt.daily_return_on_collateral >= min_daily_pct:
                result.append(opt)
        return result


if alpaca:
    options_fetcher = OptionsDataFetcher(alpaca)
    print("[OptionsDataFetcher] OK — Alpaca options client ready.")
    try:
        p = equity_fetcher.get_current_prices([config.ticker_universe[0]]) if equity_fetcher else {}
        price = p.get(config.ticker_universe[0], 0)
        if price:
            puts = options_fetcher.get_puts_chain(config.ticker_universe[0], price, config)
            print(f"  Sample: {config.ticker_universe[0]} @ ${price:.2f} -> {len(puts)} puts (min daily return filter).")
    except Exception as e:
        print(f"  Sample get_puts_chain error: {e}")
else:
    options_fetcher = None
    print("[OptionsDataFetcher] Skipped — Alpaca not configured.")

## 4. Signal Layer (Equity & Option Filters)

In [8]:
def compute_sma(series: pd.Series, period: int) -> pd.Series:
    return series.rolling(period, min_periods=period).mean()

def compute_rsi(series: pd.Series, period: int = 14) -> pd.Series:
    delta = series.diff()
    gain = delta.where(delta > 0, 0.0)
    loss = (-delta).where(delta < 0, 0.0)
    avg_gain = gain.rolling(period, min_periods=period).mean()
    avg_loss = loss.rolling(period, min_periods=period).mean()
    rs = avg_gain / avg_loss.replace(0, np.nan)
    return 100 - (100 / (1 + rs))

def compute_bb(series: pd.Series, period: int = 50, num_std: float = 1.0) -> Tuple[pd.Series, pd.Series]:
    sma = series.rolling(period, min_periods=period).mean()
    std = series.rolling(period, min_periods=period).std()
    upper = sma + num_std * std
    return sma, upper

def equity_passes_filter(close_series: pd.Series, cfg: StrategyConfig) -> bool:
    """Price between SMAs and BB upper; SMA50 uptrend (last 3 days); RSI in [rsi_lower, rsi_upper]."""
    if close_series is None or len(close_series) < max(cfg.bb_period, cfg.sma_periods[-1], cfg.rsi_period):
        return False
    price = close_series.iloc[-1]
    sma8 = compute_sma(close_series, 8)
    sma20 = compute_sma(close_series, 20)
    sma50 = compute_sma(close_series, 50)
    _, bb_upper = compute_bb(close_series, cfg.bb_period, cfg.bb_std)
    rsi = compute_rsi(close_series, cfg.rsi_period)
    n = len(close_series) - 1
    if n < cfg.bb_period or pd.isna(sma50.iloc[n]) or pd.isna(bb_upper.iloc[n]) or pd.isna(rsi.iloc[n]):
        return False
    if price >= bb_upper.iloc[n] or price <= sma8.iloc[n]:
        return False
    # SMA50 uptrend: price > SMA50 and SMA50(-3) < SMA50(-2) < SMA50(-1) roughly
    lb = cfg.sma_trend_lookback
    if n - lb < 0:
        return False
    sma50_vals = [sma50.iloc[n - i] for i in range(lb + 1)]
    if any(pd.isna(v) for v in sma50_vals):
        return False
    if price <= sma50.iloc[n]:
        return False
    if not (sma50_vals[-1] >= sma50_vals[-2] >= sma50_vals[-3]):
        return False
    rsi_val = rsi.iloc[n]
    if rsi_val < cfg.rsi_lower or rsi_val > cfg.rsi_upper:
        return False
    return True

def option_passes_filter(opt: OptionContract, cfg: StrategyConfig) -> bool:
    """DTE in range; strike in [min_strike_pct, max_strike_pct] of price; delta in [delta_min, delta_max]."""
    if opt.dte < cfg.min_dte or opt.dte > cfg.max_dte:
        return False
    strike_pct = opt.strike / opt.stock_price if opt.stock_price > 0 else 0
    if strike_pct < cfg.min_strike_pct or strike_pct > cfg.max_strike_pct:
        return False
    da = opt.delta_abs
    if da is None:
        return False
    if da < cfg.delta_min or da > cfg.delta_max:
        return False
    return True

print("[Signal layer] equity_passes_filter and option_passes_filter defined.")
if equity_fetcher and config.ticker_universe:
    try:
        ch = equity_fetcher.get_close_history(config.ticker_universe[:3], config.history_days)
        investible_test = [s for s, ser in ch.items() if equity_passes_filter(ser, config)]
        print(f"  Investible symbols (equity filter): {investible_test}")
    except Exception as e:
        print(f"  Equity filter test error: {e}")

## 5. Execution Layer (Orders & Position Tracking)

In [9]:
# Active CSP positions: list of dicts with symbol, option_symbol, strike, entry_price, entry_delta, entry_vix, entry_stock_price, qty
active_portfolio: List[dict] = []


def get_investible_symbols(close_history: Dict[str, pd.Series], cfg: StrategyConfig) -> List[str]:
    """Symbols that pass equity technical filter."""
    return [s for s, series in close_history.items() if equity_passes_filter(series, cfg)]


def select_best_put(puts: List[OptionContract], cfg: StrategyConfig) -> Optional[OptionContract]:
    """Among puts passing option filter, pick one (e.g. by daily return)."""
    filtered = [p for p in puts if option_passes_filter(p, cfg)]
    if not filtered:
        return None
    return max(filtered, key=lambda p: p.daily_return_on_collateral)


def place_csp_order(opt: OptionContract, qty: int, cfg: StrategyConfig) -> Optional[dict]:
    """Place sell-put order (market or limit per config). Returns order info or None. No-op if dry_run."""
    if not alpaca or cfg.dry_run:
        print(f"[DRY RUN] Would sell {qty} {opt.symbol} @ {'limit' if cfg.use_limit_entry else 'market'}")
        return {'dry_run': True, 'symbol': opt.symbol, 'qty': qty}
    try:
        if cfg.use_limit_entry:
            req = LimitOrderRequest(symbol=opt.symbol, qty=qty, side=OrderSide.SELL, time_in_force=TimeInForce.DAY, limit_price=round(opt.bid, 2))
        else:
            req = MarketOrderRequest(symbol=opt.symbol, qty=qty, side=OrderSide.SELL, time_in_force=TimeInForce.DAY)
        order = alpaca.trading_client.submit_order(req)
        return {'id': str(order.id), 'symbol': opt.symbol, 'qty': qty, 'side': 'sell'}
    except Exception as e:
        print("Order failed:", e)
        return None


def close_option_position(option_symbol: str, qty: int, cfg: StrategyConfig, limit_price: Optional[float] = None) -> Optional[dict]:
    """Buy to close (market or limit)."""
    if not alpaca or cfg.dry_run:
        print(f"[DRY RUN] Would buy to close {qty} {option_symbol}")
        return {'dry_run': True}
    try:
        if cfg.use_limit_exit and limit_price is not None:
            req = LimitOrderRequest(symbol=option_symbol, qty=qty, side=OrderSide.BUY, time_in_force=TimeInForce.DAY, limit_price=round(limit_price, 2))
        else:
            req = MarketOrderRequest(symbol=option_symbol, qty=qty, side=OrderSide.BUY, time_in_force=TimeInForce.DAY)
        order = alpaca.trading_client.submit_order(req)
        return {'id': str(order.id)}
    except Exception as e:
        print("Close order failed:", e)
        return None

print("[Execution] place_csp_order, close_option_position, get_investible_symbols, select_best_put loaded.")
print(f"  active_portfolio size: {len(active_portfolio)}")

In [10]:
def check_stop_loss(pos: dict, cfg: StrategyConfig, current_stock: float, current_delta: Optional[float],
                    current_vix: float) -> Tuple[bool, str]:
    """Returns (triggered, reason)."""
    if current_stock <= (1 - cfg.stock_drop_stop_pct) * pos.get('entry_stock_price', float('inf')):
        return True, "stock_drop_5pct"
    if current_vix >= pos.get('entry_vix', 0) * cfg.vix_spike_multiplier:
        return True, "vix_spike"
    if current_delta is not None and pos.get('entry_delta') is not None:
        if abs(current_delta) >= abs(pos['entry_delta']) * cfg.delta_stop_multiplier:
            return True, "delta_doubled"
    return False, ""


def check_stop_gain(pos: dict, cfg: StrategyConfig, current_premium: float, current_dte: int) -> Tuple[bool, str]:
    """Early exit if premium decay ahead of schedule by buffer."""
    entry_dte = pos.get('entry_dte')
    if entry_dte is None or entry_dte <= 0:
        return False, ""
    # Expected remaining value: (1/entry_dte)*current_dte of original; exit if current >= that * (1 - buffer)
    expected_remaining_frac = current_dte / entry_dte
    entry_premium = pos.get('entry_premium')
    if entry_premium is None or entry_premium <= 0:
        return False, ""
    target_remaining = entry_premium * expected_remaining_frac * (1 - cfg.early_exit_buffer)
    if current_premium <= target_remaining:
        return True, "early_exit_ahead_of_schedule"
    return False, ""


def get_open_option_positions() -> List[dict]:
    """Fetch current option positions from Alpaca (symbol, qty, side, etc.)."""
    if not alpaca:
        return []
    try:
        positions = alpaca.trading_client.get_all_positions()
        return [{'symbol': p.symbol, 'qty': int(float(p.qty)), 'side': p.side} for p in positions if hasattr(p, 'symbol') and p.symbol and p.symbol != p.symbol.upper()[:4]]
    except Exception as e:
        print("Get positions error:", e)
        return []

print("[Risk] check_stop_loss, check_stop_gain, get_open_option_positions loaded.")

## 6. One-Iteration (Dry-Run or Live) & Polling Loop

In [11]:
def run_one_iteration(cfg: StrategyConfig) -> dict:
    """One pass: refresh VIX/equity/options, check exits, then try to open new CSPs. Returns summary."""
    summary = {"vix": None, "deployable_cash": None, "exits": [], "entries": [], "errors": []}
    if not alpaca and not cfg.dry_run:
        summary["errors"].append("Alpaca not configured")
        return summary
    try:
        vix = vix_fetcher.get_current_vix()
        summary["vix"] = vix
        deployable = cfg.get_deployable_cash(vix)
        summary["deployable_cash"] = deployable
    except Exception as e:
        summary["errors"].append(f"VIX: {e}")
        return summary

    # Exit checks for active_portfolio
    session_vix = vix_fetcher.get_last_session().get('open', vix)
    vix_stop = vix_fetcher.check_vix_stop_loss(session_vix, cfg.vix_spike_multiplier)
    if vix_stop["triggered"]:
        print(f"  [run_one_iteration] VIX stop triggered (current={vix_stop['current_vix']:.2f} >= {vix_stop['threshold']:.2f}) — closing all.")
        for pos in list(active_portfolio):
            reason = "vix_stop"
            close_option_position(pos["option_symbol"], pos["qty"], cfg, limit_price=None)
            active_portfolio.remove(pos)
            summary["exits"].append({"symbol": pos["option_symbol"], "reason": reason})

    for pos in list(active_portfolio):
        try:
            stock_price = equity_fetcher.get_current_prices([pos["underlying"]]).get(pos["underlying"]) if equity_fetcher else None
            if stock_price is None:
                continue
            puts = options_fetcher.get_puts_chain(pos["underlying"], stock_price, cfg) if options_fetcher else []
            current_delta = None
            current_ask = None
            current_dte = None
            for p in puts:
                if p.symbol == pos["option_symbol"]:
                    current_delta = p.delta
                    current_ask = p.ask
                    current_dte = p.dte
                    break
            stop_loss, reason_sl = check_stop_loss(pos, cfg, stock_price, current_delta, vix)
            stop_gain, reason_sg = check_stop_gain(pos, cfg, current_ask or 0, current_dte or 0)
            if stop_loss or stop_gain:
                reason = reason_sl or reason_sg
                print(f"  [run_one_iteration] Exit {pos['option_symbol']}: {reason}")
                close_option_position(pos["option_symbol"], pos["qty"], cfg, limit_price=current_ask)
                active_portfolio.remove(pos)
                summary["exits"].append({"symbol": pos["option_symbol"], "reason": reason})
        except Exception as e:
            summary["errors"].append(f"Exit check {pos.get('option_symbol')}: {e}")

    # New entries: investible symbols, then best put per symbol
    if deployable <= 0 or not equity_fetcher or not options_fetcher:
        if deployable <= 0:
            print("  [run_one_iteration] No deployable cash — skipping new entries.")
        return summary
    try:
        close_history = equity_fetcher.get_close_history(cfg.ticker_universe, cfg.history_days)
        investible = get_investible_symbols(close_history, cfg)
        print(f"  [run_one_iteration] Investible symbols (equity filter): {investible}")
        current_prices = equity_fetcher.get_current_prices(investible) if investible else {}
        already_underlyings = {p["underlying"] for p in active_portfolio}
        for sym in investible:
            if sym in already_underlyings:
                continue
            price = current_prices.get(sym)
            if not price or price <= 0:
                continue
            puts = options_fetcher.get_puts_chain(sym, price, cfg)
            best = select_best_put(puts, cfg)
            if best is None:
                print(f"  [run_one_iteration] {sym}: no qualifying put (puts count={len(puts)}).")
                continue
            collateral = best.collateral_required
            if collateral > deployable:
                print(f"  [run_one_iteration] {sym}: collateral ${collateral:,.0f} > deployable ${deployable:,.0f} — skip.")
                continue
            qty = max(1, int(deployable // collateral))
            order_result = place_csp_order(best, qty, cfg)
            if order_result and not order_result.get("dry_run"):
                active_portfolio.append({
                    "option_symbol": best.symbol, "underlying": sym, "strike": best.strike, "qty": qty,
                    "entry_price": best.mid, "entry_delta": best.delta, "entry_vix": vix,
                    "entry_stock_price": price, "entry_dte": best.dte, "entry_premium": best.premium,
                })
                summary["entries"].append({"symbol": best.symbol, "qty": qty})
                deployable -= collateral * qty
    except Exception as e:
        summary["errors"].append(f"Entries: {e}")
    if summary.get("deployable_cash") is not None and summary.get("vix") is not None:
        print(f"  [run_one_iteration] VIX={summary['vix']:.2f} deployable=${summary['deployable_cash']:,.0f} exits={len(summary.get('exits', []))} entries={len(summary.get('entries', []))} errors={len(summary.get('errors', []))}")
    return summary

In [12]:
import time

def run_polling_loop(cfg: StrategyConfig, max_iterations: Optional[int] = None):
    """Run polling loop: each iteration runs run_one_iteration then sleeps poll_interval_seconds."""
    it = 0
    while max_iterations is None or it < max_iterations:
        it += 1
        print(f"--- Iteration {it} ---")
        s = run_one_iteration(cfg)
        print("VIX:", s.get("vix"), "| Deployable:", s.get("deployable_cash"), "| Exits:", len(s.get("exits", [])), "| Entries:", len(s.get("entries", [])))
        if s.get("errors"):
            print("Errors:", s["errors"])
        if s.get("exits"):
            print("Exits:", s["exits"])
        if s.get("entries"):
            print("Entries:", s["entries"])
        print("Active positions:", len(active_portfolio))
        time.sleep(cfg.poll_interval_seconds)

## 7. Test: One-Iteration Dry-Run

In [13]:
# Ensure dry_run is True to avoid real orders
config.dry_run = True
summary = run_one_iteration(config)
print("Summary:", summary)
print(f"  VIX: {summary.get('vix')} | Deployable cash: {summary.get('deployable_cash')}")
print(f"  Exits: {summary.get('exits', [])} | Entries: {summary.get('entries', [])}")
print(f"  Errors: {summary.get('errors', [])}")
print(f"  Active portfolio count: {len(active_portfolio)}")

Summary: {'vix': 17.440000534057617, 'deployable_cash': 40000.0, 'exits': [], 'entries': [], 'errors': []}


In [14]:
# Optional: validate data fetchers and dry-run candidate (run with Alpaca configured)
print("[Validation] Data fetchers and candidate check:")
if alpaca and equity_fetcher and options_fetcher:
    close_hist = equity_fetcher.get_close_history(config.ticker_universe[:3], config.history_days)
    investible = get_investible_symbols(close_hist, config)
    print(f"  Investible symbols (equity filter): {investible}")
    if investible:
        sym = investible[0]
        price = equity_fetcher.get_current_prices([sym])[sym]
        puts = options_fetcher.get_puts_chain(sym, price, config)
        best = select_best_put(puts, config)
        print(f"  Sample best put for {sym}: {best.symbol if best else None} | delta_abs: {getattr(best, 'delta_abs', None) if best else None} | daily_return_on_collateral: {best.daily_return_on_collateral:.4%}" if best else f"  No best put for {sym}")
    else:
        print("  No symbols passed equity filter.")
else:
    print("  Skipped — Alpaca or fetchers not available.")

Investible symbols (equity filter): []


To run live (paper) trading: set `config.dry_run = False` and run the polling loop. Order types: `config.use_limit_entry` / `config.use_limit_exit` control limit vs market for entry and exit.